In [11]:
import pandas as pd
import os 
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


def calculate_time_domain_features(df):
    time_domain_features = {}
    for axis in ['X', 'Y', 'Z']:
        axis_data = df[f'accelerometer_{axis}']
        time_domain_features[f'Mean_{axis}'] = axis_data.mean()
        time_domain_features[f'Std_{axis}'] = axis_data.std()
        time_domain_features[f'Max_{axis}'] = axis_data.max()
        time_domain_features[f'Min_{axis}'] = axis_data.min()
    return time_domain_features


def load_data(zip_filename):
    data = []
    labels = []
    
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        files = zip_ref.namelist()
        
        for file in files:
            if file.endswith('.csv'):
                label = os.path.basename(file).split('-')[0]
                labels.append(label)
                df = pd.read_csv(zip_ref.open(file))
                time_domain_features = calculate_time_domain_features(df)
                data.append(time_domain_features)
    
    return pd.DataFrame(data), labels


def main():
    X, y = load_data('homework.zip')
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    svm_model = SVC(kernel='linear')
    rf_model = RandomForestClassifier(n_estimators=100)

    svm_model.fit(X_train, y_train)
    rf_model.fit(X_train, y_train)

    svm_pred = svm_model.predict(X_test)
    rf_pred = rf_model.predict(X_test)

    print("Wyniki SVM:")
    print(classification_report(y_test, svm_pred, target_names=label_encoder.classes_))
    
    print("Wyniki lasu losowego:")
    print(classification_report(y_test, rf_pred, target_names=label_encoder.classes_))


if __name__ == "__main__":
    main()


Wyniki SVM:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       220
     running       1.00      1.00      1.00       689
      stairs       0.81      0.70      0.75        30
     walking       0.97      0.99      0.98       354

    accuracy                           0.99      1293
   macro avg       0.95      0.92      0.93      1293
weighted avg       0.99      0.99      0.99      1293

Wyniki lasu losowego:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       220
     running       1.00      1.00      1.00       689
      stairs       0.97      0.97      0.97        30
     walking       1.00      1.00      1.00       354

    accuracy                           1.00      1293
   macro avg       0.99      0.99      0.99      1293
weighted avg       1.00      1.00      1.00      1293

